# STT(whisper 모델)를 통해 audio 파일 -> 텍스트 파일로 변환

In [ ]:
#!pip install --upgrade git+https://github.com/openai/whisper.git
#pip install pydub

In [ ]:
import re
import os
import csv
import torch
import whisper
import pandas as pd
from pydub import AudioSegment

torch.cuda.empty_cache()
model = whisper.load_model('large', device=('cuda:0'))

designated_month = "202310"  # directory 이름. 월별 directory 내에는 음성파일이 들어가 있음.
f_list = os.listdir(f'/content/{designated_month}')

folder_list = [folder_name for folder_name in f_list if os.path.isdir(f'/content/{designated_month}/{folder_name}')]

folder_list = sorted(folder_list)

days = folder_list
len_days = len(days)

def split_audio(file_path, segment_length=180000): #오디오 파일 3분 단위로 분할
    audio = AudioSegment.from_file(file_path)
    segments = []
    for i in range(0, len(audio), segment_length):
        segments.append(audio[i:i + segment_length])
    return segments

for day in days:
    df = pd.DataFrame(columns=["variables1", "variables2"])

    file_path = f"/content/{designated_month}/{day}"
    file_list = os.listdir(file_path)
    file_length = len(file_list)
    transcripts = []
    number_list = []

    for i in range(file_length):
        file_name = file_list[i]

        name, ext = os.path.splitext(file_name)
        tokens = name.split("_")

        audio_segments = split_audio(f"/content/{designated_month}/{day}/{file_name}")

        for idx, segment in enumerate(audio_segments):
            segment_name = f"{name}_part{idx}.wav"
            number_list.append([tokens[1] + f"_part{idx}"])
            segment.export(segment_name, format="wav")
            try:
                transcribed = model.transcribe(segment_name, fp16=False)  # 음성 파일을 텍스트로 변환
                transcripts.append(transcribed['text'])
            except:
                transcripts.append(" ")
                continue

    variables1_df = pd.DataFrame(number_list, columns=['variables1'])
    variables2_df = pd.DataFrame(transcripts, columns=['variables2'])
    df = pd.concat([variables1_df, variables2_df], axis=1)
    globals()[f"df_{day}"] = df
    globals()[f"df_{day}"].to_csv(f"df_{day}.csv", index=False)
    print(f"{day}_transcription done")
print("DONE!")

In [ ]:
# CSV 파일 경로
file_path = f'df_{day}.csv'

# CSV 파일 읽기
df = pd.read_csv(file_path)
print("원본 데이터:")
print(df.head())

# variables1 컬럼에서 파일 번호를 추출하고, variables2 컬럼에서 텍스트를 추출
df['file_number'] = df['variables1'].apply(lambda x: x.split('_part')[0])
df_grouped = df.groupby('file_number')['variables2'].apply(' '.join).reset_index()

# 결과를 확인
print("변환된 데이터:")
print(df_grouped.head())

# 변환된 데이터를 CSV 파일로 저장
output_path = f'df_{day}_combined.csv'
df_grouped.to_csv(output_path, index=False)
print(f"결합된 데이터가 {output_path}에 저장되었습니다.")

# 추출한 텍스트에서 변수 추출

In [ ]:
#pip install openai

In [ ]:
import openai
import numpy as np
import pandas as pd
import re
import json

def get_completion(prompt, model='gpt-3.5-turbo'):
    message = [{'role': 'user', 'content': prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=message,
        temperature=0,
    )
    return response.choices[0].message['content']


In [ ]:
list_of_feature = {
    "간병 시작일": "",
    "단축 가능성": "",
    "연장 가능성": "",
    "단축/연장 가능성": "",
    "병원 이름": "",
    "병동": "",
    "층": "",
    "호실": "",
    "환자 이름": "",
    "성별": "",
    "나이": "",
    "몸무게": "",
    "병명(진단명)": "",
    "병실 유형": "",
    "입원 경로": "",
    "코로나": "",
    "CRE": "",
    "VRE": "",
    "결핵": "",
    "환자의식상태": "",
    "환자위생관리": "",
    "환자식사보조": "",
    "환자화장실 도움": "",
    "환자거동 도움": "",
    "피딩": "",
    "석션": "",
    "욕창": "",
    "재활치료": "",
    "장루": "",
    "투석": "",
    "치매": "",
    "섬망": "",
    "전신마비": "",
    "하반신마비": "",
    "편마비": "",
    "기타증상": "",
    "요구사항": "",
    "선호간병인성별": "",
    "선호간병인국적": "",
    "선호간병인연령": ""
}

In [ ]:
openai.api_key = #api key
data_path = 'data/df_1004_combined.csv'
text_datas = pd.read_csv(f'{data_path}')
phone_nums = list(text_datas['file_number'].astype('str'))
texts = text_datas['variables2']

phone_num_ls = []


In [ ]:
data = []

for phone_num, text in zip(phone_nums, texts):
    prompt = f"""
    너가 해야할 일은 내가 제시해주는 통화 내용의 특성을 추출하는거야.
    추출해야할 특성들은 내가 제시해줄꺼야.
    dictionary를 제시해줄테니 그 key를 특성으로 받아서 추출한 뒤 value에다가 입력해.
    만약 해당 특성이 통화내용에 드러나지 않으면 value를 'blank'로 채워.
    특성을 추출한 뒤에는 다시 value가 채워진 dictionary를 반환해줘.

    통화 내용: {text}

    추출할 특성들:
    """
    for key in list_of_feature.keys():
        prompt += f"{key}\n"

    try:
        response = get_completion(prompt)
        print("Response:", response)  # 응답 출력
        response_lines = response.split('\n')  # 응답을 줄 단위로 분할

        if not response_lines or response_lines == ['']:  # 응답이 비어있거나 줄바꿈이 없는 경우 확인
            print("No valid lines in response.")
            continue

        for line in response_lines:
            print("Line:", line)  # 각 줄을 출력하여 확인
            for key in list_of_feature.keys():
                if key in line:
                    value = line.split(':', 1)[1].strip()
                    list_of_feature[key] = value if value else "blank"
                    break

        data.append(list(list_of_feature.values()))
    except Exception as e:
        print(f"Error processing phone number {phone_num}: {e}")
        data.append(["blank"] * len(list_of_feature))
    phone_num_ls.append(phone_num)

df = pd.DataFrame(data, columns=list_of_feature.keys(), index = phone_num_ls)
df.to_csv('extracted_features_gpt_3.5.csv', index=True, encoding="utf-8-sig")
print(df)

# 모델 생성을 위한 데이터 전처리

In [ ]:
data = pd.read_csv('data/extracted_features_gpt_3.5.csv')
data = data.rename(columns={'Unnamed: 0':'전화번호'})
data = data.drop(["간병 시작일",
                   '연장 가능성',
                   "단축 가능성",
                   '병동', '층', '호실', '병실 유형', '입원 경로'], axis = 1)

binary = ['코로나', 'CRE', 'VRE', '결핵', '환자위생관리', '환자식사보조', '환자화장실 도움',
       '환자거동 도움', '피딩', '석션', '욕창', '재활치료', '장루', '투석', '치매', '섬망', '전신마비',
       '하반신마비', '편마비', '기타증상', '요구사항']
non_bin = ['병원 이름', '환자 이름', '성별' ,'나이', '몸무게',
           '병명(진단명)', '환자의식상태', '선호간병인성별',
           '선호간병인국적', '선호간병인연령', '단축/연장 가능성']
for column in binary:
    data[column] = data[column].apply(lambda x: 0 if x == "'blank'," else 1)
for column in non_bin:
    data[column] = data[column].apply(lambda x: 0 if x == "'blank'," else x)

In [ ]:
cg = pd.read_csv('data/간병목록-간병인 Data(만족도 0~5).csv', encoding='cp949')

In [ ]:
cg = cg.loc[0:9,]
cg = cg.drop(['보호자','환자', '기간','연장','성별','연령','체중',
              '배변_기저귀','배변_도움필요','배변_소변줄','배변_스스로가능',
              '의식_무의식','의식_소통원활','의식_이해가능','의식_표현가능',
              '거동_거동가능','거동_부축필요','거동_자세변경','거동_휠체어',
              '의식_뱃줄','의식_일반식','의식_콧줄'
             ], axis = 1)

In [ ]:
data1 = pd.concat([cg, data], axis = 1)
data1.to_csv('data/간병인_환자_유준영.csv',index=True, encoding="utf-8-sig")

# RandomForest, KNN, SVM을 이용한 만족도 예측모델 생성

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt

In [ ]:
file_path = 'data/간병인_환자_유준영.csv' ##모델 생성에
data = pd.read_csv(file_path)

In [ ]:
relevant_columns = data[['경력(개월)', '성별.1', '만족도',
    '자격증_간병인자격증', '자격증_간호조무사', '자격증_노인심리상담사', '자격증_실버케어지도사',
    '자격증_요양보호사', '자격증_호스피스', '자격증_활동보조인',
    '선호_감염성질환', '선호_경증환자', '선호_고액환자', '선호_남성환자', '선호_단기환자',
    '선호_석션', '선호_섬망환자', '선호_여성환자', '선호_와상환자', '선호_장기환자',
    '선호_장루', '선호_정형외과', '선호_중증환자', '선호_치매환자', '선호_편마비환자', '선호_피딩',
    '기피_격리환자', '기피_경증환자', '기피_경추환자', '기피_과체중환자', '기피_남성환자',
    '기피_단기환자', '기피_석션환자', '기피_섬망환자', '기피_수면장애환자', '기피_암환자',
    '기피_잠못자는환자', '기피_장루', '기피_중증환자', '기피_치매환자', '기피_파킨슨병',
    '기피_폐질환', '기피_피딩',
    '치매', '섬망', '전신마비', '하반신마비', '편마비', '기타증상', '요구사항'
]]

In [ ]:
#모델학습
X = relevant_columns.drop('만족도', axis = 1)
y = relevant_columns['만족도']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
RF_model = RandomForestRegressor(random_state=42)
RF_model.fit(X_train, y_train)
#성능평가
y_pred = RF_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

metrics = {
    'Metric': ['Mean Squared Error (MSE)', 'Mean Absolute Error (MAE)', 'R-squared (R2)'],
    'Value': [mse, mae, r2]
}
metrics_df = pd.DataFrame(metrics)
metrics_df

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR

In [ ]:
#모델학습
knn_model = KNeighborsRegressor()
knn_model.fit(X_train, y_train)
y_pred_knn = knn_model.predict(X_test)

svm_model = SVR()
svm_model.fit(X_train, y_train)
y_pred_svm = svm_model.predict(X_test)
#성능평가
mse_knn = mean_squared_error(y_test, y_pred_knn)
mae_knn = mean_absolute_error(y_test, y_pred_knn)
r2_knn = r2_score(y_test, y_pred_knn)

mse_svm = mean_squared_error(y_test, y_pred_svm)
mae_svm = mean_absolute_error(y_test, y_pred_svm)
r2_svm = r2_score(y_test, y_pred_svm)

metrics_data_knn = {
    'Metric': ['Mean Squared Error (MSE)', 'Mean Absolute Error (MAE)', 'R-squared (R2)'],
    'Value': [mse_knn, mae_knn, r2_knn]
}
metrics_df_knn = pd.DataFrame(metrics_data_knn)
metrics_df_knn

metrics_data_svm = {
    'Metric': ['Mean Squared Error (MSE)', 'Mean Absolute Error (MAE)', 'R-squared (R2)'],
    'Value': [mse_svm, mae_svm, r2_svm]
}
metrics_df_svm = pd.DataFrame(metrics_data_svm)
metrics_df_svm